# Punto 1 – Modelado Predictivo de Ventas (Retail)

**Objetivo:**
- Construir 3 modelos *baseline* (Regresión Lineal, Random Forest, Gradient Boosting o XGBoost) con validación cruzada (k=5) y métricas **R²** y **MAPE**.
- Seleccionar el mejor y luego **optimizar Gradient Boosting** (XGBoost si está disponible; si no, `GradientBoostingRegressor`).

**Entradas esperadas:**
- `/mnt/data/Punto1.tiendas_100.csv` (train con `ventas_m24`).
- `/mnt/data/Punto1.tiendas_10_no_target.csv` (10 nuevas tiendas sin target).

**Salidas:**
- Tabla comparativa de resultados baseline (CV=5).
- Mejor modelo tras *random search* y su tabla de **importancia de variables**.
- **Predicciones** para las 10 tiendas nuevas (archivo CSV listo para el Google Sheet del parcial).


In [32]:
import os, warnings
warnings.filterwarnings('ignore')
import numpy as np, pandas as pd
from sklearn.model_selection import KFold, cross_validate, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Intentar XGBoost; si no existe, se usará GradientBoostingRegressor
try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except Exception:
    HAS_XGB = False

DATA_TRAIN = '../data/clean/Punto1.tiendas_100_clean.csv'
DATA_NEW   = '../data/clean/Punto1.tiendas_10_no_target_clean.csv'
TARGET = 'ventas_m24'

print('XGBoost disponible?', HAS_XGB)
print('Archivos presentes en /mnt/data:', os.listdir('/mnt/data') if os.path.exists('/mnt/data') else 'No existe /mnt/data')

XGBoost disponible? True
Archivos presentes en /mnt/data: No existe /mnt/data


## 1) Carga de datos y definición de *features*
Detectamos automáticamente columnas numéricas y categóricas (por tipo).

In [33]:
assert os.path.exists(DATA_TRAIN), 'No se encontró el archivo de entrenamiento: ' + DATA_TRAIN 
assert os.path.exists(DATA_NEW),   'No se encontró el archivo de nuevas tiendas: ' + DATA_NEW

df_train = pd.read_csv(DATA_TRAIN)
df_new   = pd.read_csv(DATA_NEW)

assert TARGET in df_train.columns, f'La columna target {TARGET} no está en el train'

cat_cols = [c for c in df_train.columns if df_train[c].dtype == 'object' and c != TARGET]
num_cols = [c for c in df_train.columns if c not in cat_cols + [TARGET]]

print('Shape train:', df_train.shape)
print('Shape nuevas tiendas:', df_new.shape)
print('Numéricas:', num_cols)
print('Categóricas:', cat_cols)

X = df_train.drop(columns=[TARGET])
y = df_train[TARGET].values

Shape train: (100, 27)
Shape nuevas tiendas: (10, 25)
Numéricas: ['lat', 'lon', 'pop_100m', 'pop_300m', 'pop_500m', 'commerces', 'gas_stations', 'malls', 'foot_traffic', 'car_traffic', 'socio_level', 'viviendas_100m', 'oficinas_100m', 'viviendas_pobreza', 'competencia', 'tiendas_peq', 'pop_total', 'pop_comp_100m', 'pop_comp_300m', 'pop_comp_500m', 'pop_comp_total', 'oficinas_comp', 'viviendas_comp', 'is_outlier']
Categóricas: ['Tienda', 'store_cat']


## 2) Métricas y *preprocessing*
- **MAPE** como métrica principal (evitando división por cero).
- Dos *preprocessors*: uno con *scaling* para modelo lineal; otro sin *scaling* para árboles.

In [34]:
def mape(y_true, y_pred):
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    eps = 1e-8
    denom = np.where(np.abs(y_true) < eps, eps, np.abs(y_true))
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100.0

mape_scorer = make_scorer(mape, greater_is_better=False)

preprocess_linear = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
    ]), num_cols),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ]), cat_cols),
])

preprocess_tree = ColumnTransformer([
    ('num', SimpleImputer(strategy='median'), num_cols),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ]), cat_cols),
])

cv = KFold(n_splits=5, shuffle=True, random_state=42)

- Entonces como `sklearn`no trae por defecto en `mape` toca calcularlo.
- A pesar que el el `punto1_eda.ipynb` nos dimos cuenta que habian sensibilidades en ciertos casos y poca correlación lineal, definimos que NO ibamos a usar **modelos lineaes**, igual los probaremos para tener un pequeño feedback del rendimiento (intuimos que peor que los arboles) en el problema.
- Usaremos varios modelos de **arboles** para evitar esta sensibilidad.

## 3) Modelos *baseline* (Regresión Lineal, Random Forest, Gradient Boosting)
Calculamos **R²** y **MAPE** en CV=5 para cada modelo.

In [35]:
results = []

# 1. Linear Regression (log-transform target)
lin = Pipeline([
  ('pre', preprocess_linear),
  ('reg', LinearRegression()),
])
lin_ttr = TransformedTargetRegressor(regressor=lin, func=np.log1p, inverse_func=np.expm1)
scores_lin = cross_validate(lin_ttr, X, y, cv=cv, scoring={'R2': 'r2', 'MAPE': mape_scorer}, n_jobs=-1)
results.append({
  'model': 'LinearRegression (log-target)',
  'R2_mean': np.mean(scores_lin['test_R2']),
  'R2_std':  np.std(scores_lin['test_R2']),
  'MAPE_mean': -np.mean(scores_lin['test_MAPE']),
  'MAPE_std':  np.std(scores_lin['test_MAPE']),
})

# 2. Random Forest
rf = Pipeline([
  ('pre', preprocess_tree),
  ('rf', RandomForestRegressor(n_estimators=300, min_samples_leaf=3, random_state=42, n_jobs=-1)),
])
scores_rf = cross_validate(rf, X, y, cv=cv, scoring={'R2': 'r2', 'MAPE': mape_scorer}, n_jobs=-1)
results.append({
  'model': 'RandomForestRegressor',
  'R2_mean': np.mean(scores_rf['test_R2']),
  'R2_std':  np.std(scores_rf['test_R2']),
  'MAPE_mean': -np.mean(scores_rf['test_MAPE']),
  'MAPE_std':  np.std(scores_rf['test_MAPE']),
})

# 3. Gradient Boosting (sklearn)
gbr = Pipeline([
  ('pre', preprocess_tree),
  ('gb', GradientBoostingRegressor(n_estimators=400, learning_rate=0.05, max_depth=3, subsample=0.9, random_state=42)),
])
scores_gbr = cross_validate(gbr, X, y, cv=cv, scoring={'R2': 'r2', 'MAPE': mape_scorer}, n_jobs=-1)
results.append({
  'model': 'GradientBoostingRegressor',
  'R2_mean': np.mean(scores_gbr['test_R2']),
  'R2_std':  np.std(scores_gbr['test_R2']),
  'MAPE_mean': -np.mean(scores_gbr['test_MAPE']),
  'MAPE_std':  np.std(scores_gbr['test_MAPE']),
})

# 4. XGBoost (si está disponible)
if HAS_XGB:
  xgb = Pipeline([
    ('pre', preprocess_tree),
    ('gb', XGBRegressor(
      n_estimators=400, learning_rate=0.05, max_depth=4,
      subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
      random_state=42, n_jobs=-1, tree_method='hist'
    )),
  ])
  scores_xgb = cross_validate(xgb, X, y, cv=cv, scoring={'R2': 'r2', 'MAPE': mape_scorer}, n_jobs=-1)
  results.append({
    'model': 'XGBRegressor',
    'R2_mean': np.mean(scores_xgb['test_R2']),
    'R2_std':  np.std(scores_xgb['test_R2']),
    'MAPE_mean': -np.mean(scores_xgb['test_MAPE']),
    'MAPE_std':  np.std(scores_xgb['test_MAPE']),
  })

df_results_baseline = pd.DataFrame(results).sort_values('MAPE_mean')
df_results_baseline

,model,R2_mean,R2_std,MAPE_mean,MAPE_std
3,XGBRegressor,0.860105,0.060509,15.956722,2.504691
1,RandomForestRegressor,0.827256,0.084777,16.086279,3.486338
2,GradientBoostingRegressor,0.857887,0.087602,16.250015,3.388854
0,LinearRegression (log-target),-1.400612,3.336512,27.566383,5.103824


### Linear:
- Se hizo una **trasformacion logaritmica** ya que en `punto1_eda.ipynb`se vio un poco de sesgo a la izquierda. Tambien porque se mantuvieron los outlayers.
- Se uso `expm1`para devolver las predicciones a las escalas originales y asi entender.

### Arboles
- Se usaron respectivamente cada parametro intentado dar una variabilidad inicial para revisar como cada modelo puede rendir, basadon en generalidad.
  - `n_estimators` numero de arboles
  - `min_samples_leaf`tamaño minimo de hojas
  - `n_jobs` nucleos
  - `learning_rate`aprendizaje de arboles
  - `max_depth` proffundidad de los arboles
  - `subsample`aleatoridad
  - `colsample_bytree` columnas usadas y aleatoridad
  - `tree_method` probar este hiperparametro


### General:
- `mean`representa el promedio entre los 5 folds del CV. (Entrevar 5 veces con distintas particiones en **train/test**)
- `std`variabilidad entre los fols. ¿Que tan estables son los resultados?


## Conclusiones para la optimización de los modelos

- Los resultados nos confirman lo encontrado en el `punto1_edad.ipynb`:
  - **Las relaciones NO son lineales**, Tambien hay **Outlayers estructurales** y posible **colinealidad**.
  - Se evidencia que los modelos **NO** lineales son más robustos
  - Las variables poblacionales + viviendas son las más correlacionadas con ventas
- Aunque Random Forest y GB también son competitivos, XGB muestra más consistencia.
- Los std de R² y MAPE están en rangos manejables (2–3.5% MAPE, 0.06–0.08 R²).
- Eso significa que los resultados no dependen mucho del fold → buena generalización.


## 4) Optimización de Gradient Boosting (*RandomizedSearchCV*)
Se optimizan hiperparámetros clave para mejorar MAPE y controlar sobreajuste.

In [36]:
if HAS_XGB:
    gb_search = Pipeline([
        ('pre', preprocess_tree),
        ('gb', XGBRegressor(random_state=42, n_jobs=-1, tree_method='hist')),
    ])
    param_dist = {
        'gb__n_estimators': [200, 400, 600, 800, 1000],
        'gb__max_depth': [2, 3, 4, 5, 6],
        'gb__learning_rate': np.linspace(0.01, 0.2, 10),
        'gb__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'gb__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
        'gb__reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0],
    }
else:
    gb_search = Pipeline([
        ('pre', preprocess_tree),
        ('gb', GradientBoostingRegressor(random_state=42)),
    ])
    param_dist = {
        'gb__n_estimators': [200, 400, 600, 800, 1000],
        'gb__max_depth': [2, 3, 4, 5],
        'gb__learning_rate': np.linspace(0.01, 0.2, 10),
        'gb__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'gb__max_features': [None, 'sqrt', 0.6, 0.8],
        'gb__min_samples_leaf': [1, 2, 3, 5, 10],
    }

rand_search = RandomizedSearchCV(
    estimator=gb_search,
    param_distributions=param_dist,
    n_iter=40, cv=cv,
    scoring={'R2': 'r2', 'MAPE': mape_scorer},
    refit='MAPE', random_state=42, n_jobs=-1, verbose=1
)
rand_search.fit(X, y)
best_model = rand_search.best_estimator_
best_mape = -rand_search.best_score_
best_params = rand_search.best_params_

print('Mejor MAPE (CV):', round(best_mape, 3))
print('Mejores parámetros:', best_params)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Mejor MAPE (CV): 14.05
Mejores parámetros: {'gb__subsample': 1.0, 'gb__reg_lambda': 1.0, 'gb__n_estimators': 200, 'gb__max_depth': 2, 'gb__learning_rate': np.float64(0.03111111111111111), 'gb__colsample_bytree': 0.6}


Usando estos datos se hizo un modelo que explora las variables.

Vamos a usar los datos obtenidos de varios modelos para hacer un **modelo final**

In [37]:
# ===================== MODELO FINAL (XGB con log-target) =====================
import numpy as np, pandas as pd, os, json
from sklearn.model_selection import RepeatedKFold, cross_validate
from sklearn.compose import TransformedTargetRegressor
from xgboost import XGBRegressor

RANDOM_STATE = 42
rkf = RepeatedKFold(n_splits=5, n_repeats=3, random_state=RANDOM_STATE)

def mape(y_true, y_pred):
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    eps = 1e-8
    denom = np.where(np.abs(y_true) < eps, eps, np.abs(y_true))
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100.0

from sklearn.metrics import make_scorer
mape_scorer = make_scorer(mape, greater_is_better=False)

scoring = {
    'MAPE': mape_scorer,
    'R2': 'r2',
    'MAE': 'neg_mean_absolute_error',
    'RMSE': 'neg_root_mean_squared_error',
    'EVS': 'explained_variance'
}

# --- Modelo con hiperparámetros finales (de tu búsqueda) ---
xgb_best = TransformedTargetRegressor(
    regressor=Pipeline(steps=[
        ('pre', preprocess_tree),
        ('gb', XGBRegressor(
            objective='reg:squarederror',
            random_state=RANDOM_STATE,
            n_jobs=-1,
            tree_method='hist',
            n_estimators=1800,
            learning_rate=0.014827586206896552,
            max_depth=5,
            min_child_weight=2,
            subsample=0.6,
            colsample_bytree=0.7,
            gamma=0.1,
            reg_alpha=0.0,
            reg_lambda=1.0,
        ))
    ]),
    func=np.log1p, inverse_func=np.expm1
)

# ===================== 1) EVALUACIÓN CV + OOF =====================
# cross_validate para métricas agregadas
cv_rep = cross_validate(
    xgb_best, X, y, cv=rkf, scoring=scoring,
    return_estimator=False, return_train_score=True, n_jobs=-1, verbose=0
)

def cv_summary(cv_rep, metric):
    test = np.array(cv_rep[f'test_{metric}']); tr = np.array(cv_rep[f'train_{metric}'])
    # invertir signo para MAE/RMSE
    sign = -1 if metric in ['MAE', 'RMSE'] else 1
    return {
        'mean_test': float(sign*np.mean(test)),
        'std_test':  float(np.std(test)),
        'mean_train': float(sign*np.mean(tr)),
        'gap_train_test': float(sign*np.mean(tr) - sign*np.mean(test))
    }

summary = pd.DataFrame({
    'MAPE(%)': cv_summary(cv_rep, 'MAPE'),
    'R2':      cv_summary(cv_rep, 'R2'),
    'MAE':     cv_summary(cv_rep, 'MAE'),
    'RMSE':    cv_summary(cv_rep, 'RMSE'),
    'EVS':     cv_summary(cv_rep, 'EVS'),
}).T
print('=== CV Summary ===')
print(summary)

# OOF (out-of-fold) para tener “predicciones de test” internas
# Nota: usamos Predicción K-fold manual porque cross_val_predict NO soporta TransformedTargetRegressor bien con pipelines complejos
from sklearn.base import clone
oof_pred = np.zeros(len(X), dtype=float)
oof_idx = np.zeros(len(X), dtype=int)
k = 0
for train_idx, test_idx in rkf.split(X, y):
    model_k = clone(xgb_best)
    model_k.fit(X.iloc[train_idx], y[train_idx])
    oof_pred[test_idx] = model_k.predict(X.iloc[test_idx])
    oof_idx[test_idx] = k
    k += 1

oof_df = pd.DataFrame({
    'Tienda': df_train['Tienda'].values if 'Tienda' in df_train.columns else np.arange(len(df_train)),
    'y_true': y,
    'y_pred_oof': oof_pred,
    'fold': oof_idx
})
oof_metrics = {
    'MAPE(%)': float(mape(oof_df['y_true'], oof_df['y_pred_oof'])),
    'R2': float(__import__('sklearn.metrics').metrics.r2_score(oof_df['y_true'], oof_df['y_pred_oof'])),
}
print('\n=== OOF Metrics ===', oof_metrics)

# ===================== 2) ENTRENAR EN TODO EL TRAIN =====================
xgb_best.fit(X, y)

# ===================== 3) IMPORTANCIA DE VARIABLES =====================
def get_feature_names(preprocessor, num_cols, cat_cols):
    num_names = list(num_cols)
    ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
    cat_names = list(ohe.get_feature_names_out(cat_cols))
    return num_names + cat_names

# extraer importancias desde el paso 'gb' del pipeline interno
pre_fitted = xgb_best.regressor_.named_steps['pre']
feat_names = get_feature_names(pre_fitted, num_cols, cat_cols)
gb_step = xgb_best.regressor_.named_steps['gb']
importancias = pd.DataFrame({
    'feature': feat_names,
    'importance': gb_step.feature_importances_
}).sort_values('importance', ascending=False)
print('\nTop-20 features:')
print(importancias.head(20))

# ===================== 4) PREDICCIONES NUEVAS TIENDAS =====================
# Alinear columnas de df_new con X (por si falta/ sobra algo)
def align_columns(train_df, new_df, drop_target=True):
    cols = [c for c in train_df.columns if (c != TARGET if drop_target else True)]
    for c in cols:
        if c not in new_df.columns:
            new_df[c] = np.nan
    extra = [c for c in new_df.columns if c not in cols]
    if extra:
        new_df = new_df.drop(columns=extra)
    return new_df[cols]

X_new = align_columns(df_train, df_new.copy(), drop_target=True)
pred_new = xgb_best.predict(X_new)

pred_new_df = pd.DataFrame({
    'Tienda': df_new['Tienda'].values if 'Tienda' in df_new.columns else np.arange(len(df_new)),
    'pred_ventas_m24': pred_new
})

# ===================== 5) EXPORTS =====================
os.makedirs('outputs_p1', exist_ok=True)
summary.to_csv('outputs_p1/cv_summary_metrics.csv', index=True)
oof_df.to_csv('outputs_p1/oof_predictions.csv', index=False)
importancias.to_csv('outputs_p1/feature_importances.csv', index=False)
pred_new_df.to_csv('outputs_p1/pred_nuevas_tiendas.csv', index=False)

# (Opcional) métricas CV completas por fold
cv_full = pd.DataFrame({
    'test_MAPE': cv_rep['test_MAPE'],
    'test_R2': cv_rep['test_R2'],
    'test_MAE': cv_rep['test_MAE'],
    'test_RMSE': cv_rep['test_RMSE'],
    'test_EVS': cv_rep['test_EVS'],
    'train_MAPE': cv_rep['train_MAPE'],
    'train_R2': cv_rep['train_R2'],
    'train_MAE': cv_rep['train_MAE'],
    'train_RMSE': cv_rep['train_RMSE'],
    'train_EVS': cv_rep['train_EVS'],
})
cv_full.to_csv('outputs_p1/cv_fold_metrics.csv', index=False)

print('\nArchivos generados en ./outputs_p1')


=== CV Summary ===
          mean_test    std_test  mean_train  gap_train_test
MAPE(%)  -15.260897    2.959712   -6.635495        8.625402
R2         0.831609    0.083187    0.936009        0.104400
MAE      375.934776  107.824141  177.858353     -198.076423
RMSE     595.604687  313.701711  379.948424     -215.656263
EVS        0.846488    0.073848    0.937544        0.091056

=== OOF Metrics === {'MAPE(%)': 15.682066535636885, 'R2': 0.7903493432659061}

Top-20 features:
               feature  importance
2             pop_100m    0.183730
16           pop_total    0.176060
3             pop_300m    0.075128
4             pop_500m    0.071495
11      viviendas_100m    0.070505
23          is_outlier    0.038115
22      viviendas_comp    0.036510
13   viviendas_pobreza    0.035041
1                  lon    0.028591
17       pop_comp_100m    0.026411
124  store_cat_express    0.023830
5            commerces    0.022703
10         socio_level    0.022190
6         gas_stations    0.022013

### Evidenciamos:

- MAPE de **-15.260897** con un STD **2.959712** un error controlado en nuevos datos, es decir que generaliza bien el model.
- Tengo un **R²** con un 0.83 y un 0.08 de **std**. Pude explicar el 80% de la varianza de ventas
- Hay un pequeño overlifting MAPE train ≈ 6.6% vs test ≈ 15%.

## 5) Importancia de variables del mejor modelo
Se extraen importancias del modelo (árboles) o coeficientes (si fuera lineal).

In [40]:
importance_df_path = 'outputs_p1/feature_importances.csv'

important_df = pd.read_csv(importance_df_path)

important_df.head(10), sum(important_df['importance'])

(             feature  importance
 0           pop_100m    0.183730
 1          pop_total    0.176060
 2           pop_300m    0.075128
 3           pop_500m    0.071495
 4     viviendas_100m    0.070505
 5         is_outlier    0.038115
 6     viviendas_comp    0.036510
 7  viviendas_pobreza    0.035041
 8                lon    0.028591
 9      pop_comp_100m    0.026411,
 0.999999969)

### Evidenciamos:

- Población (100m, 300m, 500m, total) → explican la mayor parte de la señal.
- Viviendas (100m, comp, pobreza) → alta correlación con ventas.
- Flag is_outlier → captura hipermercados/tiendas premium con ventas excepcionales.
- Competencia ajustada (viviendas_comp, oficinas_comp) → refuerza el hallazgo del EDA: la competencia sola no explica, pero sí en interacción con densidad.
- Categoría de tienda (store_cat_express) y malls influyen en dispersión.
- Tráfico y comercios → aportan, aunque en menor medida.

## 6) Predicciones para las 10 nuevas tiendas
Se aplica el pipeline completo `best_model` sobre `df_new` para obtener `ventas_m24_pred`.

In [41]:
# Predicciones para nuevas tiendas (modelo final XGB + log-target)

from sklearn.compose import TransformedTargetRegressor
from xgboost import XGBRegressor

# Modelo final (mismos hiperparámetros ganadores)
xgb_best = TransformedTargetRegressor(
    regressor=Pipeline(steps=[
        ('pre', preprocess_tree),
        ('gb', XGBRegressor(
            objective='reg:squarederror',
            random_state=42,
            n_jobs=-1,
            tree_method='hist',
            n_estimators=1800,
            learning_rate=0.014827586206896552,
            max_depth=5,
            min_child_weight=2,
            subsample=0.6,
            colsample_bytree=0.7,
            gamma=0.1,
            reg_alpha=0.0,
            reg_lambda=1.0,
        ))
    ]),
    func=np.log1p, inverse_func=np.expm1
)

# Entrenar en TODO el train
xgb_best.fit(X, y)

# Alinear columnas de nuevas tiendas al esquema de X (por si falta/sobra algo)
def align_columns(train_df, new_df, drop_target=True):
    cols = [c for c in train_df.columns if (c != TARGET if drop_target else True)]
    for c in cols:
        if c not in new_df.columns:
            new_df[c] = np.nan
    extra = [c for c in new_df.columns if c not in cols]
    if extra:
        new_df = new_df.drop(columns=extra)
    return new_df[cols]

X_new = align_columns(df_train, df_new.copy(), drop_target=True)

# Predecir y exportar
pred_new = xgb_best.predict(X_new)
pred_new_df = pd.DataFrame({
    'Tienda': df_new['Tienda'].values if 'Tienda' in df_new.columns else np.arange(len(df_new)),
    'pred_ventas_m24': pred_new
})

os.makedirs('outputs_p1', exist_ok=True)
pred_new_df.to_csv('outputs_p1/pred_nuevas_tiendas.csv', index=False)
print('Guardado: outputs_p1/pred_nuevas_tiendas.csv')
print(pred_new_df.head(10))


Guardado: outputs_p1/pred_nuevas_tiendas.csv
       Tienda  pred_ventas_m24
0  Tienda_101      3019.705078
1  Tienda_102      1918.226685
2  Tienda_103      1256.778809
3  Tienda_104      1598.940552
4  Tienda_105      5360.463379
5  Tienda_106      1957.577393
6  Tienda_107      1123.873535
7  Tienda_108       959.754272
8  Tienda_109       867.478821
9  Tienda_110      3680.788818


## 7) Entregable

In [43]:
import pandas as pd, os, json, joblib, sys
from datetime import datetime

DOC_ESTUDIANTE = "1234567890"      # <-- edita
NOMBRE_ESTUDIANTE = "Andrés Yañez" # <-- edita

# Carga de artefactos generados antes
summary = pd.read_csv('outputs_p1/cv_summary_metrics.csv', index_col=0)
oof = pd.read_csv('outputs_p1/oof_predictions.csv')
importancias = pd.read_csv('outputs_p1/feature_importances.csv')
pred_new = pd.read_csv('outputs_p1/pred_nuevas_tiendas.csv')

# Explicación breve (edita si quieres)
explicacion = [
 "Modelo: XGBoost con transformación logarítmica del target.",
 "CV (5x3): MAPE ≈ 15.3% (std ≈ 3 p.p.) y R² ≈ 0.83 (std ≈ 0.08).",
 "Principales impulsores de ventas: población total y por radios, viviendas;",
 "influyen además categoría de tienda y densidad ajustada por competencia.",
 "Competencia aislada tiene baja señal; al ajustarla por población sí aporta.",
 "El modelo generaliza bien; gap train-test esperado por outliers estructurales."
]
df_explicacion = pd.DataFrame({"Explicación breve del modelo e interpretación": explicacion})

# Encabezado con datos del estudiante
encabezado = pd.DataFrame({
    "Documento": [DOC_ESTUDIANTE],
    "Nombre": [NOMBRE_ESTUDIANTE],
    "Fecha_export": [datetime.now().strftime("%Y-%m-%d %H:%M")]
})

# Ensamble del archivo final
os.makedirs('outputs_p1', exist_ok=True)
ruta_xlsx = 'outputs_p1/ENTREGABLE_Punto1.xlsx'
with pd.ExcelWriter(ruta_xlsx, engine='xlsxwriter') as writer:
    encabezado.to_excel(writer, sheet_name='Punto 1', index=False, startrow=0)
    pred_new.to_excel(writer, sheet_name='Punto 1', index=False, startrow=3)
    importancias.to_excel(writer, sheet_name='Punto 1', index=False, startrow=3+len(pred_new)+3)
    df_explicacion.to_excel(writer, sheet_name='Punto 1', index=False, startrow=3+len(pred_new)+3+len(importancias)+2)
    # Métricas detalladas en otra hoja (opcional)
    summary.to_excel(writer, sheet_name='Metricas_CV')
    oof[['Tienda','y_true','y_pred_oof','fold']].to_excel(writer, sheet_name='OOF', index=False)

print("Listo: outputs_p1/ENTREGABLE_Punto1.xlsx (importa este archivo a Google Sheets y renombra la hoja si es necesario).")

# (Opcional) Guardar el modelo final para reproducibilidad
joblib.dump(xgb_best, 'outputs_p1/modelo_xgb_log_final.joblib')
meta = {
  "python": sys.version,
  "libs": {"pandas": pd.__version__},
  "fecha": datetime.now().isoformat(),
  "params": {
    "n_estimators": 1800, "learning_rate": 0.014827586206896552, "max_depth": 5,
    "min_child_weight": 2, "subsample": 0.6, "colsample_bytree": 0.7,
    "gamma": 0.1, "reg_alpha": 0.0, "reg_lambda": 1.0, "log_target": True
  }
}
with open('outputs_p1/metadata.json','w') as f: json.dump(meta, f, indent=2)


Listo: outputs_p1/ENTREGABLE_Punto1.xlsx (importa este archivo a Google Sheets y renombra la hoja si es necesario).
